# Circuit learning module: Pennylane with SPSA

This code is inspired by [Quanthoven](https://github.com/CQCL/Quanthoven/blob/main/experiment.ipynb) and [Pennylane demo](https://pennylane.ai/qml/demos/spsa.html).

In [50]:
import json
import os
import glob
import warnings
from pathlib import Path
import numpy as np
import pickle
import pennylane as qml
from sympy import default_sort_key
import torch
from discopy.quantum.pennylane import to_pennylane, PennyLaneCircuit
from inspect import signature
from noisyopt import minimizeSPSA
from utils import transform_into_pennylane_circuits, read_diagrams, get_symbols, create_labeled_classes, multi_class_loss

this_folder = os.path.abspath(os.getcwd())
nshot = 1000

## Read circuit data

We read the circuits from the pickled files.

In [17]:
# Select workload
workload = "execution_time"
#workload = "cardinality"

# Select workload size
#workload_size = "small"
#workload_size = "medium"
#workload_size = "large"
workload_size = "main"

classification = 1
layers = 1
single_qubit_params = 3
n_wire_count = 1

# Access the selected circuits
path_name = this_folder + "//simplified-JOB-diagrams//"\
            + workload + "//" + workload_size + "//circuits//"\
            + str(classification) + "//" + str(layers) + "_layer//"\
           + str(single_qubit_params) + "_single_qubit_params//" + str(n_wire_count)\
            + "_n_wire_count//"

In [18]:
training_circuits_paths = glob.glob(path_name + "training//[0-9]*.p")
validation_circuits_paths = glob.glob(path_name + "validation//[0-9]*.p")
test_circuits_paths = glob.glob(path_name + "test//[0-9]*.p")

In [19]:
training_circuits = read_diagrams(training_circuits_paths)
validation_circuits = read_diagrams(validation_circuits_paths)
test_circuits = read_diagrams(test_circuits_paths)

In [24]:
n_qubits = 15
dev = qml.device("default.qubit", wires=n_qubits, shots=nshot)

qml_training_circuits, train_symbols = transform_into_pennylane_circuits(training_circuits, n_qubits, dev)
qml_test_circuits, test_symbols = transform_into_pennylane_circuits(test_circuits, n_qubits, dev)
qml_validation_circuits, val_symbols = transform_into_pennylane_circuits(validation_circuits, n_qubits, dev)
#qml_circuits = qml_training_circuits + qml_test_circuits + qml_validation_circuits

In [26]:
#training_qnodes = qml.QNodeCollection([c["qml_circuit"] for c in qml_training_circuits.values()])
#validation_qnodes = qml.QNodeCollection([c["qml_circuit"] for c in qml_validation_circuits.values()])
#test_qnodes = qml.QNodeCollection([c["qml_circuit"] for c in qml_test_circuits.values()])

In [27]:
print("Number of training circuits: ", len(qml_training_circuits))
print("Number of validation circuits: ", len(qml_validation_circuits))
print("Number of test circuits: ", len(qml_test_circuits))

Number of training circuits:  448
Number of validation circuits:  113
Number of test circuits:  112


## Read training and test data

In [30]:
training_data, test_data, validation_data = None, None, None
data_path = this_folder + "//data//" + workload + "//" + workload_size + "//"

with open(data_path + "training_data.json", "r") as inputfile:
    training_data = json.load(inputfile)['training_data']
with open(data_path + "test_data.json", "r") as inputfile:
    test_data = json.load(inputfile)['test_data']
with open(data_path + "validation_data.json", "r") as inputfile:
    validation_data = json.load(inputfile)['validation_data']

training_data_labels = create_labeled_classes(training_data, classification, workload)
test_data_labels = create_labeled_classes(test_data, classification, workload)
validation_data_labels = create_labeled_classes(validation_data, classification, workload)

## Post-selection

In [ ]:
def post_select_circuit_samples(circuit_samples, n_qubits, post_selection):
    selected_samples = []
    post_select_array = np.array([0]*(n_qubits - post_selection))
    for circuit_sample in circuit_samples:
        if np.array_equal(circuit_sample[post_selection - 1 :-1], post_select_array):
            res = circuit_sample[0:post_selection]
            selected_samples.append(res)
            #if circuit_sample[0] == 1:
            #    selected_samples.append(1)
            #else:
            #    selected_samples.append(0)
    return selected_samples

## Cost function

In [ ]:
def make_pred_fn(circuits):
    def predict(params):
        outputs = Circuit.eval(*(c(*params) for c in circuit_fns), backend = backend)
        res = []
        
        for output in outputs:
            predictions = np.abs(output.array) + 1e-9
            ratio = predictions / predictions.sum()
            res.append(ratio)
            
        return np.array(res)
    return predict

In [46]:
def acc_from_dict(dict_predictions, dict_labels):
    total_acc = 0
    for query_id in dict_predictions:
        y_meas = np.array(dict_predictions[query_id]).flatten()
        max_index = np.argmax(y_meas)
        total_acc += int(int(dict_labels[query_id][max_index]) == 1)
    return total_acc / len(dict_predictions)


def loss_from_dict(dict_predictions, dict_labels):
    total_loss = 0
    for query_id in dict_predictions:
        x = np.array(dict_labels[query_id])
        y_pred = np.array(dict_predictions[query_id]).flatten()
        total_loss += -np.sum(x * np.log(y_pred)) / len(x)
    return total_loss

In [7]:
def make_cost_fn(qnodes, labels):    
    def cost_spsa(params, **kwargs):
        predictions = {}
        for q in qnodes:
            circuit = qnodes[q]
            measurement = circuit(params)
            post_selected_samples = post_select_circuit_samples(measurement)
            counts = collections.Counter(post_selected_result)
            res = {}
            for s in collections.OrderedDict(sorted(counts.items())):
                res[s] = counts[s]/len(post_selected_samples)
            predictions[q] = res
        
        cost = loss_from_dict(predictions, labels) #-np.sum(labels * np.log(predictions)) / len(labels)  # binary cross-entropy loss
        costs.append(cost)

        accuracy = acc_from_dict(predictions, labels) #np.sum(np.round(predictions) == labels) / len(labels) / 2  # half due to double-counting
        accuracies.append(accuracy)
        
        return cost
    
    costs, accuracies = [], []
    return cost_spsa, costs, accuracies

## SPSA minimization

In [8]:
BATCH_SIZE = 32
EPOCHS = 200
SEED = 0

rng = np.random.default_rng(SEED)
init_params_spsa = np.array(rng.random(len(symbols)))
numpy.random.seed(SEED)

train_cost_fn, train_costs, train_accs = make_cost_fn(training_qnodes, training_data_labels_l)
#dev_cost_fn, dev_costs, dev_accs = make_cost_fn(dev_pred_fn, dev_labels)

# Evaluate the initial cost
cost_store_spsa = [train_cost_fn(init_params_spsa)]
#print(init_params_spsa)
#print()
#print(training_data_labels_l)
#print()
#print(cost_store_spsa)
device_execs_spsa = [0]

def callback_fn(xk):
    cost_val = train_cost_fn(xk)
    cost_store_spsa.append(cost_val)

    # We've evaluated the cost function, let's make up for that
    num_executions = int(dev.num_executions / 2)
    device_execs_spsa.append(num_executions)

    iteration_num = len(cost_store_spsa)
    print(
            f"Params = {xk}, "
            f"Iteration = {iteration_num}, "
            f"Number of device executions = {num_executions}, "
            f"Cost = {cost_val}"
        )

In [ ]:
result = minimizeSPSA(train_cost_fn, x0=init_params_spsa.copy(), a=0.91, c=0.12, niter=EPOCHS, paired=False, callback=callback_fn)

#res = minimizeSPSA(
#    cost_spsa,
#    x0=init_params_spsa.copy(),
#    niter=EPOCHS,
#    paired=False,
#    c=0.15,
#    a=0.2,
#    callback=callback_fn,
#)

Params = [0.60104737 0.2338724  0.00505921 ... 0.89903452 0.84539286 0.47479219], Iteration = 2, Number of device executions = 50, Cost = 0.7120762862749391
Params = [ 0.69515929  0.13976048 -0.08905271 ...  0.8049226   0.93950478
  0.38068027], Iteration = 3, Number of device executions = 87, Cost = 0.6615862146685579
Params = [ 0.78384029  0.05107949 -0.1777337  ...  0.71624161  1.02818577
  0.46936127], Iteration = 4, Number of device executions = 125, Cost = 0.7460381406386465
Params = [ 0.78922026  0.04569951 -0.17235373 ...  0.72162158  1.03356575
  0.46398129], Iteration = 5, Number of device executions = 162, Cost = 0.7107609971686729
Params = [ 0.78303381  0.05188596 -0.16616728 ...  0.72780803  1.0273793
  0.47016774], Iteration = 6, Number of device executions = 200, Cost = 0.6668101021206293
Params = [ 0.76154107  0.03039322 -0.14467454 ...  0.74930077  1.00588656
  0.448675  ], Iteration = 7, Number of device executions = 237, Cost = 0.707839039079418
Params = [ 0.73653022